In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader
from typing import Tuple
from collections import defaultdict

In [2]:
df = pd.read_csv('./datasets/ml-1m/ratings.dat', delimiter='::', header=None)
df.columns = ['UserID', 'MovieID', 'Rating', 'Timestamp']
df = df.drop(columns=['Timestamp'])
df.head()

c:\Users\mokom\Anaconda3\envs\data_science\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,UserID,MovieID,Rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [3]:
df.MovieID.max()

3952

In [4]:
rating_matrix = df.pivot(index='UserID', columns='MovieID', values='Rating')
n_users, n_movies = rating_matrix.shape
print(f'num of users: {n_users}  num of items: {n_movies}')

sparcity = rating_matrix.notna().sum().sum() / (n_users * n_movies)
print(f'Sparcity: {sparcity:0.2%}')

rating_matrix

num of users: 6040  num of items: 3706
Sparcity: 4.47%


MovieID,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
UserID,,,,,,,,,,,,,,,,,,,,,
1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,NaN,NaN,NaN,2.0,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
class MatrixFactorization(nn.Module):
    def __init__(self, n_users, n_item, k=20):
        super().__init__()
        self.user_factors = nn.Embedding(n_users, k, sparse=True)
        self.item_factors = nn.Embedding(n_item, k, sparse=True)

    def forward(self, user, item):
        return (self.user_factors(user) * self.item_factors(item)).sum(1)

In [6]:
class MovieLens1mDataset(Dataset):
    USER_ID = 0
    MOVIE_ID = 1
    RATING = 2

    def __init__(self, rating_path: str) -> None:
        super().__init__()
        self.df = pd.read_csv(rating_path, delimiter='::', header=None)
        self.df.columns = ['UserID', 'MovieID', 'Rating', 'Timestamp']
        self.df = self.df.drop(columns=['Timestamp'])

    def __getitem__(self, index: int) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        values = self.df.iloc[index].values
        # minus 1 to use user_id and movie_id for matrix indexes
        user_id = values[self.USER_ID] - 1
        movie_id = values[self.MOVIE_ID] - 1
        target = np.float32(values[self.RATING])
        return user_id, movie_id, target

    def __len__(self) -> int:
        return len(self.df)

In [7]:
dataset = MovieLens1mDataset('./datasets/ml-1m/ratings.dat')
n_train = int(len(dataset)*0.7)
n_val = len(dataset) - n_train
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [n_train, n_val])

c:\Users\mokom\Anaconda3\envs\data_science\lib\site-packages\ipykernel_launcher.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [8]:
BATCH_SIZE = 64

train_dataloader = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True,
    pin_memory=True
)

val_dataloader = DataLoader(
    val_dataset, 
    batch_size=1, 
    shuffle=True,
    pin_memory=True # faster read
)

dataloaders = dict(train=train_dataloader, val=val_dataloader)

In [9]:
def train_model(model, dataloaders: dict, n_epoch: int, optimizer, criterion):
    loss_results = defaultdict(list)

    for epoch in range(n_epoch):
        loss_per_epoch = dict(train=0, val=0)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
        
            for users, items, targets in tqdm(dataloaders[phase]):
                # 勾配を初期化
                optimizer.zero_grad()
                
                # 学習時のみ勾配を計算
                with torch.set_grad_enabled(phase == 'train'):
                    preds = model(users, items)
                    loss = criterion(preds, targets)
                    loss_per_epoch[phase] += loss

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    
        loss_results[phase].append(loss_per_epoch[phase])
        
        print(f"[epoch {epoch+1}] train loss: {loss_per_epoch['train']}   val loss: {loss_per_epoch['val']}")

In [10]:
n_users, n_items = dataset.df.UserID.max(), dataset.df.MovieID.max()
matrix_factorization = MatrixFactorization(n_users, n_items, k=20)
criterion = nn.MSELoss()
optimizer = optim.SparseAdam(matrix_factorization.parameters(), lr=1e-2)
n_epoch = 10

train_model(matrix_factorization, dataloaders, n_epoch, optimizer, criterion)

  0%|          | 1/10940 [00:00<32:45,  5.57it/s]

[epoch 1] train loss: 137863.203125   val loss: 730286.75


  0%|          | 3/10940 [00:00<07:00, 26.01it/s]

[epoch 2] train loss: 14481.7509765625   val loss: 413263.6875


  0%|          | 2/10940 [00:00<09:36, 18.97it/s]

...

[epoch 8] train loss: 8393.1181640625   val loss: 318977.65625


  0%|          | 7/10940 [00:00<02:50, 64.02it/s]

[epoch 9] train loss: 8240.7080078125   val loss: 316371.90625


100%|██████████| 300063/300063 [01:54<00:00, 2619.13it/s]

[epoch 10] train loss: 8115.3759765625   val loss: 314187.78125


In [25]:
matrix_factorization.eval()
users, items, targets = next(iter(dataloaders['val']))
print(f'predict: {matrix_factorization(users, items).detach().numpy()} target: {targets.numpy()}')

predict: [5.0426493] target: [5.]
